## Setup

Requires a HubSpot Private app with an API access token with the following scopes:

- `crm.objects.custom.read`
- `crm.objects.companies.read`
- `crm.schemas.contacts.read`

The token should be stored in an environment variable called `HUBSPOT_ACCESS_TOKEN`.

You can copy the sample environment file to get started; run the following command from the root of this repository:

```bash
cp .env.sample .env
```

Then open `.env` and fill in with your access token.

In [ ]:
import os

from hubspot import HubSpot
import pandas as pd

from data.utils import write_json_records


ACCESS_TOKEN = os.environ["HUBSPOT_ACCESS_TOKEN"]

hubspot = HubSpot(access_token=ACCESS_TOKEN)

In [ ]:
props_response = hubspot.crm.properties.core_api.get_all("notes", archived=False)
props_df = pd.json_normalize(props_response.to_dict(), "results")
write_json_records(props_df, "notes_props.json")

In [ ]:
notes_props = ["hs_created_by", "hs_createdate", "hs_note_body"]
notes_assoc = ["companies", "contacts", "vendors"]

response = hubspot.crm.objects.notes.basic_api.get_page(limit=10, properties=notes_props, associations=notes_assoc)

notes = pd.json_normalize(response.to_dict(), "results")
notes.info()